In [491]:
# Import Dependencies
import pandas as pd
from itertools import tee, islice, chain

In [492]:
# Read in Monthly Sales Price Data
price = pd.read_excel('Data/Med:Avg Sales Price.xls', sheet_name='Price ')
# Clean up DataFrame
price.columns = price.iloc[1]
price = price.iloc[2:]
# Remove timestamp from Period column
price['Period'] = pd.to_datetime(price['Period'], errors='coerce')
# Insert Median prices into empty Average values
price['Average'].loc[2:145] = price['Median '].loc[2:145]
# Drop Median column
price.drop(columns='Median ', inplace=True)
# Set Period as index
price.set_index('Period', inplace=True)
price = price.rename_axis('Period', axis='columns')
price.index.name = None
# Drop Null Values
price = price.dropna()
# Set a more accurate name for column
price.rename(columns={'Average': 'Avg. Price'}, inplace=True)

/var/folders/zq/0hmyg29903n1xkl833j8x3h00000gn/T/ipykernel_59160/475955272.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Average'].loc[2:145] = price['Median '].loc[2:145]


In [493]:
# Read in Monthly Sales Quantity Data
quantity = pd.read_excel('Data/Sold by Region.xls', sheet_name='Reg Sold')
# Clean up df
quantity = quantity.iloc[7:]
quantity.rename(columns={'Unnamed: 1': 'Quantity'}, inplace= True)
quantity.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], inplace= True)
# Drop nulls
quantity = quantity.dropna()
# Convert index to period format
quantity['Houses Sold by Region'] = pd.to_datetime(quantity['Houses Sold by Region'], errors='coerce')
quantity.rename(columns={'Houses Sold by Region': 'Period'}, inplace= True)
quantity.set_index('Period', inplace=True)
# Set quantity to actual values
quantity['Quantity'] = quantity['Quantity'] * 1000

In [494]:
# Merge two dataframes together
revenue = pd.concat([price, quantity], axis= 1)
# Give a title to the index column
revenue = revenue.rename_axis(columns='Period')
revenue

Period,Avg. Price,Quantity
1963-01-01,17200,42000
1963-02-01,17700,35000
1963-03-01,18200,44000
1963-04-01,18200,52000
1963-05-01,17500,58000
...,...,...
2022-01-01,501200,70000
2022-02-01,522200,71000
2022-03-01,512900,69000
2022-04-01,569500,57000


In [495]:
# Multiply price with quantity
revenue['Avg. Revenue'] = revenue['Avg. Price'] * revenue['Quantity']
revenue

Period,Avg. Price,Quantity,Avg. Revenue
1963-01-01,17200,42000,722400000
1963-02-01,17700,35000,619500000
1963-03-01,18200,44000,800800000
1963-04-01,18200,52000,946400000
1963-05-01,17500,58000,1015000000
...,...,...,...
2022-01-01,501200,70000,35084000000
2022-02-01,522200,71000,37076200000
2022-03-01,512900,69000,35390100000
2022-04-01,569500,57000,32461500000


In [496]:
# Function to access next and previous items in a for loop
# found on stack overflow: https://stackoverflow.com/questions/1011938/loop-that-also-accesses-previous-and-next-values
def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)

In [497]:
# Iterate over Revenue column to determine periods of Expansion and Recession
rev = revenue['Avg. Revenue']

expansion = []
recession = []
counter1 = 0
counter2 = 0

for previous, item, nxt in previous_and_next(rev):
    try:
        if previous < item:
            counter1 +=1
            expansion.append(counter1)
        else:
            counter1 = 0
            expansion.append(counter1)
    except:
        counter1 = 0
        expansion.append(counter1)
    try:
        if previous > item:
            counter2 +=1
            recession.append(counter2)
        else:
            counter2 = 0
            recession.append(counter2)
    except:
        counter2 = 0
        recession.append(counter2)

In [498]:
# Insert Expansion and Recession values into DF
revenue.insert(loc=3, column= '# of Expansions', value= expansion)
revenue.insert(loc=4, column= '# of Recessions', value= recession)

In [499]:
# Calculate percentage gained and lossed month over month
percent_change = []

for previous, item, nxt in previous_and_next(rev):
    try:
        result = ((item - previous)/previous)
        result = "{:.2%}".format(result)
        percent_change.append(result)
    except:
        result = 0 
        result = "{:.0%}".format(result)
        percent_change.append(result)

In [500]:
# Insert percentage change into DF
revenue.insert(loc=4, column= '% Change', value= percent_change)

In [501]:
# Iterate through df to find the bottom of the recession
recession = revenue['# of Recessions']
valley = []
for previous, item, nxt in previous_and_next(recession):
    try:
        if nxt < item > previous:
            valley.append('Yes')
        else:
            valley.append('No')
    except:
        valley.append('No')

In [502]:
# Iterate through df to find the peak of the expansion
expansion = revenue['# of Expansions']
peak = []
for previous, item, nxt in previous_and_next(expansion):
    try:
        if nxt < item > previous:
            peak.append('Yes')
        else:
            peak.append('No')
    except:
        peak.append('No')

In [503]:
# Insert peaks and valleys into DF
revenue.insert(loc=4, column='Peak', value= peak)
revenue.insert(loc=6, column='Valley', value= valley)


In [504]:
revenue['Avg. Revenue'] = revenue['Avg. Revenue']
revenue

Period,Avg. Price,Quantity,Avg. Revenue,# of Expansions,Peak,% Change,Valley,# of Recessions
1963-01-01,17200,42000,722400000,0,No,0%,No,0
1963-02-01,17700,35000,619500000,0,No,-14.24%,Yes,1
1963-03-01,18200,44000,800800000,1,No,29.27%,No,0
1963-04-01,18200,52000,946400000,2,No,18.18%,No,0
1963-05-01,17500,58000,1015000000,3,Yes,7.25%,No,0
...,...,...,...,...,...,...,...,...
2022-01-01,501200,70000,35084000000,3,No,17.14%,No,0
2022-02-01,522200,71000,37076200000,4,Yes,5.68%,No,0
2022-03-01,512900,69000,35390100000,0,No,-4.55%,No,1
2022-04-01,569500,57000,32461500000,0,No,-8.28%,No,2


In [505]:
# Slice dataframe into 10 year intervals
decades = {}
decades['revenue_0'] = revenue[0: 84]
months = 84
for i in range(1, round((len(revenue)/120)+1)):
    years = months + 120
    decades['revenue_' + str(i)] = revenue[months:years]
    months += 120

In [506]:
decades['revenue_0']

Period,Avg. Price,Quantity,Avg. Revenue,# of Expansions,Peak,% Change,Valley,# of Recessions
1963-01-01,17200,42000,722400000,0,No,0%,No,0
1963-02-01,17700,35000,619500000,0,No,-14.24%,Yes,1
1963-03-01,18200,44000,800800000,1,No,29.27%,No,0
1963-04-01,18200,52000,946400000,2,No,18.18%,No,0
1963-05-01,17500,58000,1015000000,3,Yes,7.25%,No,0
...,...,...,...,...,...,...,...,...
1969-08-01,24900,40000,996000000,0,No,-4.71%,No,3
1969-09-01,26000,33000,858000000,0,No,-13.86%,No,4
1969-10-01,24400,32000,780800000,0,No,-9.00%,No,5
1969-11-01,24900,31000,771900000,0,No,-1.14%,No,6


In [507]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
# Also set the default datetime and date formats.
writer = pd.ExcelWriter('US Housing Monthly Revenue.xlsx', engine='xlsxwriter', date_format='yyyy mm dd', datetime_format='yyyy-mm-dd')
revenue.to_excel(writer, sheet_name='Monthly Revenue', index_label='Period')
year = 1960 
for i in decades:
    decades[i].to_excel(writer, sheet_name=str(year) + "'s", index_label = 'Period')
    year += 10

In [508]:
# Get the xlsxwriter workbook and worksheets objects and set the column
# widths, to make the dates clearer.
workbook = writer.book
worksheets = {}
titles = 1960


for i in range(0, len(decades)):
    worksheets['worksheet_' + str(i)] = writer.sheets[str(titles) + "'s"].set_column(0, 8, 11)
    titles += 10

In [509]:
# Create a new chart object.
chart = workbook.add_chart({'type': 'line'})

# Add a series to the chart.
chart.add_series({'values': "='Monthly Revenue'!$D$2:$D$714"})
chart.set_size({'width': 1200, 'height': 1080})

# Insert the chart into the worksheet.
worksheets['worksheet_0'] = writer.sheets['Monthly Revenue'].insert_chart('K2', chart)

# Set column width
worksheets['worksheet_0'] = writer.sheets['Monthly Revenue'].set_column(0, 8, 11)

In [510]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()